# ✅Visualization based on pyvista

In [1]:
import openseespy.opensees as ops
import opstool as opst

Load the 3D Arch Bridge finite element model.

In [2]:
opst.load_ops_examples("SuspensionBridge")
# or your openseespy code
# your openseespy code here

## Display OpenSeesPy Geometry Model

In [3]:
ModelData = opst.GetFEMdata(results_dir="opstool_output")
ModelData.get_model_data(save_file="ModelData.hdf5")

***** Model Information *****

Number of nodes: 200

Number of elements: 343

The boundary size: X --> [-70.  70.]; Y --> [-1.75  1.75]; Z --> [-6.25 11.25]

Element types: {'ShellMITC4', 'ElasticBeam3d'}

***** END *****

Model data saved in opstool_output/ModelData.hdf5!

In [4]:
opsvis = opst.OpsVisPyvista(point_size=0, line_width=2,
                            colors_dict=None, theme="paraview",
                            color_map="coolwarm", on_notebook=False,
                            results_dir="opstool_output")

In [5]:
opsvis.model_vis(input_file="ModelData.hdf5",
                 show_node_label=False, show_ele_label=False,
                 show_local_crd=True,
                 local_crd_alpha=1.5,
                 show_fix_node=True,
                 fix_node_alpha=2.0,
                 show_constrain_dof=False,
                 label_size=15,
                 show_outline=True,
                 opacity=1.0,
                 save_fig='images/ModelVis.svg')

## Display Eigen Analysis

<div class="alert alert-info">

Note

Before performing the eigenvalue analysis, you need to ensure that the OpenSeesPy model is correct and that the mass is set.

</div>

Obtain the first 20 orders of modal data.

In [6]:
ModelData.get_eigen_data(mode_tag=20, solver="-genBandArpack",
                         save_file='EigenData.hdf5')

Using DomainModalProperties - Developed by: Massimo Petracca, Guido Camata, ASDEA Software Technology


Eigen data saved in opstool_output/EigenData.hdf5!

In [7]:
opsvis.eigen_vis(input_file='EigenData.hdf5',
                 mode_tags=[1, 20], subplots=False,
                 alpha=1.0, show_outline=False,
                 show_origin=False, opacity=1.0,
                 show_face_line=False, save_fig='images/EigenVis.svg')

Of course, *subplots* set to True will return a multi-sub plot.

In [8]:
opsvis.eigen_vis(input_file='EigenData.hdf5',
                 mode_tags=[1, 16], subplots=True,
                 alpha=1.0, show_outline=False,
                 show_origin=False, opacity=1.0,
                 show_face_line=False, save_fig='images/EigenVis2.svg')

In [9]:
opsvis.eigen_anim(input_file='EigenData.hdf5',
                  mode_tag=6, alpha=1.0,
                  show_outline=False,
                  opacity=1, framerate=3,
                  n_cycle=5,  # change steps here
                  show_face_line=True,
                  save_fig="images/EigenAnimation.gif")

## Display Response Data

In [10]:
opst.gen_grav_load(ts_tag=1, pattern_tag=1,
                   factor=-9.81, direction="Z")

In [11]:
Nsteps = 10
ops.wipeAnalysis()
ops.system('BandGeneral')
ops.constraints('Transformation')
ops.numberer('RCM')
ops.test('NormDispIncr', 1.0e-12, 10, 3)
ops.algorithm('Newton')
ops.integrator('LoadControl', 1 / Nsteps)
ops.analysis('Static')

# start analysis loop
for i in range(Nsteps):
    ok = ops.analyze(1)
    ModelData.get_resp_step()
# save all responses data after loop
ModelData.save_resp_all(save_file="RespStepData-1.hdf5")

All responses data saved in opstool_output/RespStepData-1.hdf5!

In [12]:
opsvis.deform_vis(input_file="RespStepData-1.hdf5",
                  slider=True,
                  response="disp", alpha=1.0,
                  show_outline=False, show_origin=True,
                  show_face_line=False, opacity=1,
                  save_fig="images/DefoVis.svg",
                  model_update=False)

In [13]:
opsvis.deform_anim(input_file="RespStepData-1.hdf5",
                    response="disp", alpha=1.0,
                    show_outline=False,
                    framerate=2,
                    show_face_line=False, opacity=1,
                    save_fig="images/DefoAnimation.gif",
                    model_update=False)

### Display Frame Element Response

In [14]:
opsvis.frame_resp_vis(input_file="RespStepData-1.hdf5",
                      ele_tags=None,
                      slider=True,
                      response="My",
                      show_values=False,
                      alpha=1.0,
                      opacity=1,
                      save_fig="images/FrameRespVis.svg")